In [2]:
#load the original ADR related People data
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd

df = pd.read_csv('ADR_people.csv')

df = df.drop('Unnamed: 0', axis=1)

#load the ADR - itemnumber matrix

matrix= pd.read_csv('Prior_ITEM_NUMBER_Matrix.csv')

matrix = matrix.set_index('item_number')

# Normalize the matrix
normalized_matrix = (matrix - matrix.min()) / (matrix.max() - matrix.min())

# Replace all NaN values with 0
normalized_matrix = normalized_matrix.fillna(0)

#load the ADR - itemnumber normalised matrix
# normalized_matrix = pd.read_csv('Normalised_Prior_ITEM_NUMBER_Matrix.csv')

#load the ADR list

ADR_list = ['D52.1', 'D59.0','D59.2','D61.1', 'D64.2','D68.3','D69.5','E03.2',
            'E06.4','E15','E16.0','E23.1','E24.2','E27.3','E66.1','F11', 'F13',
            'F15', 'F19','F55','G21.0','G21.1','G21.2','G24.0','G25.0','G25.1',
            'G25.4','G25.6','G44.4','G62.0','G72.0','H26.3','H40.6','H91.0','I42.7',
            'I95.2','J70.2','J70.3','J70.4','K52.1','K71.1','K71.2','K71.6','K71.9',
            'K85.3','L10.5','L23.3','L24.4','L25.1','L27','L27.0','L27.1','L27.8',
            'L27.9','L43.2','L51','L51.1','L51.2','L51.8','L51.9','L56.0','L56.1',
            'L64.0','M10.2','M32.0','M34.2','M80.4','M81.4','M83.5','M87.1','N14.0',
            'N14.1','N14.2','N14.3','N14.4','N99.0','O35.5','O74.2','O74.3','O74.4',
            'P04.0','P04.1','P04.4','P58.4','P93','P96.1','P96.2','Q86.1','Q86.2',
            'R50.2','T36','T37','T38','T39','T39.1','T40','T40.3','T41','T42','T43',
            'T44','T45','T46','T46.0','T47','T48','T49','T50','T78','T78.2','T78.3',
            'T78.4','T78.8','T78.9','T80.1','T80.2','T80.3','T80.4','T80.5','T80.6',
            'T80.8','T80.9','T88.3','T88.6','T88.7','X40','X41','X42','X43','X44',
            'X45.5','Y40','Y41','Y42','Y42.7','Y43','Y44','Y44.0','Y44.2','Y45',
            'Y45.5','Y46','Y47','Y47.1','Y48','Y48.1','Y48.2','Y48.3','Y48.4','Y49',
            'Y50','Y51','Y52','Y53','Y54','Y55','Y55.1','Y56','Y57','Y57.5','Y57.9',
            'Y58','Y59','Y59.9','Y63','Y65.1','Y69','Y88.0','Z03.6']
# Create learning dataset and labels based on the n_days to be considered
import pandas as pd

def generate_dataset(df, normalized_matrix, ADR_list, n_days):
    df['create_datetime'] = pd.to_datetime(df['create_datetime'], format='%d/%m/%y')
    df['item_number'] = df['item_number'].str.strip()

    new_dataset = pd.DataFrame()
    labels = pd.DataFrame()
    index_list = []

    for person_id in df['person_id'].unique():
        person_df = df[df['person_id'] == person_id]
        person_adrs = person_df[person_df['item_number'].isin(ADR_list)]['item_number'].unique()
        if len(person_adrs) > 0:
            for adr in person_adrs:
                adr_date = person_df[person_df['item_number'] == adr]['create_datetime'].max()
                
                prior_period_df = person_df[(person_df['create_datetime'] < adr_date) &
                            (person_df['create_datetime'] >= adr_date - pd.DateOffset(days=n_days)) &
                            ~((person_df['item_number'] == adr) & (person_df['create_datetime'] == adr_date))]

                average_vector = normalized_matrix.loc[prior_period_df['item_number']].max()

                new_dataset = pd.concat([new_dataset, average_vector.to_frame().T], ignore_index=True)

                label_vector = pd.Series([1 if label_adr == adr else 0 for label_adr in ADR_list], index=ADR_list)

                labels = pd.concat([labels, label_vector.to_frame().T], ignore_index=True)

                index_list.append(f"{person_id}_{adr}")

    new_dataset.index = labels.index = index_list
    
        # Store the index of the rows with NaN values in new_dataset
    dropped_rows_index = new_dataset[new_dataset.isna().any(axis=1)].index

    new_dataset = new_dataset.dropna(axis=0)

    # Drop the corresponding rows from the labels dataset
    labels = labels.drop(index=dropped_rows_index)

    return new_dataset, labels

def labels_aggregate(labels):
    import pandas as pd

    # Assuming your labels dataframe is named 'labels'
    unique_first_letters = set([col[0] for col in labels.columns])

    # Create a new dataframe to store the combined columns
    combined_labels = pd.DataFrame(index=labels.index)

    for letter in unique_first_letters:
        # Find the columns with the same first letter
        matching_columns = [col for col in labels.columns if col[0] == letter]

        # Combine the matching columns into a single column
        combined_labels[letter] = labels[matching_columns].any(axis=1).astype(int)

    # Print the combined labels
    return combined_labels

# List of desired columns
desired_columns = ['Y', 'T', 'R', 'K', 'D', 'I', 'L', 'G', 'X', 'Z']

In [3]:
df

,person_id,age,gender,episode_id,drg_number,drg_version,item_number,create_datetime
0,0,104,1,168951,L63B,4.2,B96,2/7/04
1,0,104,1,168951,L63B,4.2,n39,2/7/04
2,0,104,1,168951,L63B,4.2,Z86,2/7/04
3,0,104,1,195656,f62b,4.2,I25.5,20/7/05
4,0,104,1,195656,f62b,4.2,I95.9,20/7/05
...,...,...,...,...,...,...,...,...
123994,1659,45,0,846115,K04A,6.2,92514-39,29/3/18
123995,1659,45,0,846115,K04A,6.2,E66.93,29/3/18
123996,1659,45,0,846115,K04A,6.2,K52.1,29/3/18
123997,1659,45,0,846115,K04A,6.2,U73.8,29/3/18


In [4]:
# Convert the 'item_number' column to string data type, because Word2Vec model accepts list of sentences (where each sentence is a list of words)
df['item_number'] = df['item_number'].astype(str)
df['create_datetime'] = pd.to_datetime(df['create_datetime'], format='%d/%m/%y')
df['item_number'] = df['item_number'].str.strip()

# Order by 'create_datetime' and group by 'person_id'
df_sorted = df.sort_values('create_datetime').groupby('person_id')['item_number'].apply(list).reset_index(name='item_list')


In [5]:
df_sorted

,person_id,item_list
0,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I..."
1,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,..."
2,2,"[I10, I48, i74, Y92, I50, Z92, Y54, I48, Z86, ..."
3,3,"[Y458, Z867, Z8711, N1890, N058, 3044500, I10,..."
4,4,"[42731, V4581, V1259, 53540, V1082, 7872, V104..."
...,...,...
1655,1655,"[Y92.9, Y57.9, T78.0]"
1656,1656,"[13706-05, 13706-03, 96199-02, D69.5, Z29.2, 9..."
1657,1657,"[Y43.4, I10, R11, R53, G25.1, E87.5, M32.1, Y9..."
1658,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9..."


In [6]:
from gensim.models import Word2Vec

# Initialize the model
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Build the vocabulary
model.build_vocab(df_sorted['item_list'])

# Train the model
model.train(df_sorted['item_list'], total_examples=model.corpus_count, epochs=model.epochs)


(520463, 619995)

In [7]:
# Initialize new dataframe
new_df = pd.DataFrame(columns=['person_id', 'item_list', 'label'])

for idx, row in df_sorted.iterrows():
    # Find first ADR code in the item_list
    first_adr_idx = next((i for i, x in enumerate(row['item_list']) if x in ADR_list), None)
    
    if first_adr_idx is not None:
        # If an ADR is found, slice the list at the index of the first ADR
        adr_list = row['item_list'][:first_adr_idx]
        new_df = new_df.append({'person_id': row['person_id'], 'item_list': adr_list, 'label': 1}, ignore_index=True)
    
    # Add original list as well, label as '0'  
    new_df = new_df.append({'person_id': row['person_id'], 'item_list': row['item_list'], 'label': 0}, ignore_index=True)


In [8]:
new_df

,person_id,item_list,label
0,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I...",1
1,0,"[B96, Z86, n39, Z86.43, I50.1, Z95.0, I25.5, I...",0
2,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,...",1
3,1,"[5123, V1250, 8753, 4019, V1251, 412, 3123002,...",0
4,2,"[I10, I48, i74, Y92, I50, Z92]",1
...,...,...,...
3297,1657,"[Y43.4, I10, R11, R53, G25.1, E87.5, M32.1, Y9...",0
3298,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9...",1
3299,1658,"[W01.1, 50106-00, U73.9, 92514-99, 47459-01, 9...",0
3300,1659,[U73.8],1


In [9]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert item lists to vectors
def convert_to_vector(item_list):
    if len(item_list) > 0:
        return np.mean([model.wv[word] for word in item_list], axis=0)
    else:
        return np.zeros(model.vector_size)  # return an empty vector if item_list is empty

new_df['vector'] = new_df['item_list'].apply(convert_to_vector)


In [10]:
new_df['label'] = new_df['label'].astype(int)

X = np.array(new_df['vector'].tolist())
y = new_df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.6314199395770392
